In [7]:
import requests
import pandas as pd
import numpy as np
import re

# Define your API key and the stock symbol
api_key = "6OSEAGNZPLLNQJO8" # Replace with your actual API key
symbol = "SPY"  # Replace with the stock symbol of your choice
API_ENDPOINT_URL = 'https://www.alphavantage.co/query'
# Define API parameters
base_url = "https://www.alphavantage.co/query"  # Replace with the actual API endpoint URL
# regex to convert the column names ;;;;; r  signifies a raw string literal to help deal with escape char
pattern = r'^\d\._'

params = {
    "function": "TIME_SERIES_INTRADAY",
    "symbol": symbol,
    "interval": "15min",
    "apikey": api_key,
    "outputsize": "full",  # You may change this based on your requirements
    "datatype": "json",  # You can change to "csv" if needed
}

# Fetch intraday data from the API
response = requests.get(base_url, params=params)
data = response.json()  # Assuming JSON format
#print(data)

# Convert the data into a DataFrame
df = pd.DataFrame.from_dict(data["Time Series (15min)"], orient="index")

# Create a new column 'Date-Time' to store the original datetime values
df['Date-Time'] = pd.to_datetime(df.index)

# Reset the index to create a new integer index
df.reset_index(drop=True, inplace=True)
df.sort_index(ascending=True, inplace=True)
# Remove spaces from column names
df.columns = df.columns.str.replace(" ", "_")  # Replace spaces with underscores
df.rename(columns = lambda x : re.sub(pattern, '', x) ,inplace = True)  # Replace spaces with underscores
# Move 'Date-Time' column to the first position
date_time_column = df['Date-Time']
df = df.drop(columns=['Date-Time'])
df.insert(0, 'Date-Time', date_time_column)
# Store the data in an Excel sheet
output_excel_file = "spy_stock_data.xlsx"  # Choose a file name for your Excel file
df.to_excel(output_excel_file, index=False)  # Save the DataFrame to an Excel file


In [15]:
import requests
import pandas as pd
import numpy as np

# Define the file path of the Excel file
input_file = "spy_stock_data.xlsx" # Replace with the actual path to your Excel file

# Read data from the Excel file and convert it to a DataFrame
df = pd.read_excel(input_file)

# Initialize variables for tracking positions and trades
position = 0 # 0 represents no position, 1 represents long, -1 represents short
buy_price = 0
sell_price = 0
signals = []
returns = [] # To track individual trade returns
cumulative_return = 0
# Initialize lists to store individual signal components
signal_type = []
date_time = []
stock_price = []
ema9_values = [] # To track the 9EMA values
sma20_values = [] # To track the 20SMA values

# Calculate 9-day EMA and 20-day SMA
df['9EMA'] = df['close'].astype(float).ewm(span=9, adjust=False).mean()
df['20SMA'] = df['close'].astype(float).rolling(window=20).mean()

# Generate buy/sell signals and calculate returns
for i in range(20, len(df)):
  if df['close'][i] > df['9EMA'][i] and df['9EMA'][i] > df['20SMA'][i]:
    if position == 0:
       position = 1
       buy_price = df['close'][i]
       buy_dt = df['Date-Time'][i].strftime('%Y-%m-%d %H:%M:%S')
       signals.append(('Buy', df.index[i], buy_dt, buy_price))
  elif df['close'][i] < df['9EMA'][i] and df['9EMA'][i] < df['20SMA'][i]:
    if position == 1:
       position = 0
       sell_price = df['close'][i]
       sell_dt = df['Date-Time'][i].strftime('%Y-%m-%d %H:%M:%S')
       signals.append(('Sell', df.index[i], sell_dt, sell_price))
       trade_return = sell_price - buy_price
       returns.append(trade_return)
       cumulative_return += trade_return

output_excel_file = "spy_stock_data_with_ma.xlsx"  # Choose a file name for your Excel file
df.to_excel(output_excel_file, index=False)  # Save the DataFrame to an Excel file

In [47]:
import pandas as pd
import re
# Sample DataFrame
data = {
    '1._open': [1.0, 2.0, 3.0],
    '1._high': [4.0, 5.0, 6.0],
    '2._low': [7.0, 8.0, 9.0],
}

df = pd.DataFrame(data)

# Define a regex pattern to match column names
pattern = r'^\d\._'

# Use a lambda function to rename columns based on the regex pattern
df.rename(columns=lambda x: re.sub(pattern, '', x), inplace=True)

# Display the DataFrame with renamed columns
print(df)

   open  high  low
0   1.0   4.0  7.0
1   2.0   5.0  8.0
2   3.0   6.0  9.0


In [16]:
# ###########################
# Working with DAILY SPY data
# ###########################

import requests
import pandas as pd
import numpy as np
import re

# Define your API key and the stock symbol
api_key = "6OSEAGNZPLLNQJO8" # Replace with your actual API key
symbol = "SPY"  # Replace with the stock symbol of your choice
API_ENDPOINT_URL = 'https://www.alphavantage.co/query'
# Define API parameters
base_url = "https://www.alphavantage.co/query"  # Replace with the actual API endpoint URL
# regex to convert the column names ;;;;; r  signifies a raw string literal to help deal with escape char
pattern = r'^\d\._'

params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": symbol,
    "apikey": api_key,
    "outputsize": "full",  # You may change this based on your requirements
    "datatype": "json",  # You can change to "csv" if needed
}

# Fetch intraday data from the API
response = requests.get(base_url, params=params)
data = response.json()  # Assuming JSON format
#print(data)

# Convert the data into a DataFrame
df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")

# Create a new column 'Date-Time' to store the original datetime values
df['Date-Time'] = pd.to_datetime(df.index)

# Reset the index to create a new integer index
df.reset_index(drop=True, inplace=True)
df.sort_index(ascending=True, inplace=True)
# Remove spaces from column names
df.columns = df.columns.str.replace(" ", "_")  # Replace spaces with underscores
df.rename(columns = lambda x : re.sub(pattern, '', x) ,inplace = True)  # Replace spaces with underscores
# Move 'Date-Time' column to the first position
date_time_column = df['Date-Time']
df = df.drop(columns=['Date-Time'])
df.insert(0, 'Date-Time', date_time_column)
# Store the data in an Excel sheet
output_excel_file = "spy_stock_data.xlsx"  # Choose a file name for your Excel file
df.to_excel(output_excel_file, index=False)  # Save the DataFrame to an Excel file

In [17]:
# ###########################
# Working with DAILY SPY data
# ###########################

import pandas as pd
import numpy as np

# Define the file path of the Excel file
input_file = "spy_stock_data.xlsx" # Replace with the actual path to your Excel file

# Read data from the Excel file and convert it to a DataFrame
df = pd.read_excel(input_file)

# Initialize variables for tracking positions and trades
position = 0 # 0 represents no position, 1 represents long, -1 represents short
buy_price = 0
sell_price = 0
signals = []
returns = [] # To track individual trade returns
cumulative_return = 0
# Initialize lists to store individual signal components
signal_type = []
date_time = []
stock_price = []
ema9_values = [] # To track the 9EMA values
sma20_values = [] # To track the 20SMA values

# Calculate 9-day EMA and 20-day SMA
df['9EMA'] = df['close'].astype(float).ewm(span=9, adjust=False).mean()
df['20SMA'] = df['close'].astype(float).rolling(window=20).mean()

# Generate buy/sell signals and calculate returns
for i in range(20, len(df)):
  if df['close'][i] > df['9EMA'][i] and df['9EMA'][i] > df['20SMA'][i]:
    if position == 0:
       position = 1
       buy_price = df['close'][i]
       buy_dt = df['Date-Time'][i].strftime('%Y-%m-%d %H:%M:%S')
       signals.append(('Buy', df.index[i], buy_dt, buy_price))
  elif df['close'][i] < df['9EMA'][i] and df['9EMA'][i] < df['20SMA'][i]:
    if position == 1:
       position = 0
       sell_price = df['close'][i]
       sell_dt = df['Date-Time'][i].strftime('%Y-%m-%d %H:%M:%S')
       signals.append(('Sell', df.index[i], sell_dt, sell_price))
       trade_return = sell_price - buy_price
       returns.append(trade_return)
       cumulative_return += trade_return

output_excel_file = "spy_stock_data_with_ma.xlsx"  # Choose a file name for your Excel file
df.to_excel(output_excel_file, index=False)  # Save the DataFrame to an Excel file